In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
df=pd.read_csv("/content/train.tsv",sep='\t')
df.head()

,date,time,id,reviewer name,reviewer address,contact number,variation,verified_reviews,sentiment,feedback
0,30-Jul-18,02:16:00,5138527,Theresa Diaz,"2373 Long Mews\nSouth Lorraine, FL 83715",7445448464,Black Plus,Love my new Echo Plus,5,1
1,30-Jul-18,20:13:24,7881409,Michael Wagner,"34120 James Well Apt. 930\nSouth Kenneth, AK 5...",(202)233-6662,Configuration: Fire TV Stick,Works really well.,5,1
2,29-Jul-18,03:20:48,5658396,Erin Rodriguez,"7996 Booth Mission\nTaylorport, AL 13457",(219)882-0705,Black Show,Echo Show is said to work with certain apps bu...,2,0
3,30-Jul-18,22:48:21,4317957,Christine Perez,USS Schaefer\nFPO AA 86432,291-900-6087,Heather Gray Fabric,Very cool product. Speaker sounds good with my...,5,1
4,23-Jul-18,12:36:42,7570569,Jamie Davila,"000 Kevin Cliffs Suite 674\nBrennanview, NM 77690",9527296610,Black Plus,So far I love it. It was easy to set up - if ...,5,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              2520 non-null   object
 1   time              2520 non-null   object
 2   id                2520 non-null   int64 
 3   reviewer name     2520 non-null   object
 4   reviewer address  2520 non-null   object
 5   contact number    2520 non-null   object
 6   variation         2520 non-null   object
 7   verified_reviews  2520 non-null   object
 8   sentiment         2520 non-null   int64 
 9   feedback          2520 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 197.0+ KB


In [ ]:
df.isnull().sum()

date                0
time                0
id                  0
reviewer name       0
reviewer address    0
contact number      0
variation           0
verified_reviews    0
sentiment           0
feedback            0
dtype: int64

In [ ]:
df=df.dropna()
df.isnull().sum()

date                0
time                0
id                  0
reviewer name       0
reviewer address    0
contact number      0
variation           0
verified_reviews    0
sentiment           0
feedback            0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

y=df['sentiment']
X=df['verified_reviews']
y=y.apply(lambda x: 1 if x==5 else 0)

X_train,X_test,y_train,y_split = train_test_split(X,y,test_size=0.2,random_state=1)
X_train

761     excellent device , works great, having no prob...
1967    Part of it is my fault, I thought it could be ...
1555                          Easy set up, works perfect.
2084    What a wonderful little Alexa enabled gadget a...
1651    Works great! I recommend it to anyone who has ...
                              ...                        
960     this is so much fun to play with - I just ask ...
905                                                      
1096    Easy installation and set up. Better sound fro...
235     This thing is awesome for grocery lists, paire...
1061    It was supposed to come with a free Hue lightb...
Name: verified_reviews, Length: 2016, dtype: object

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
# tokenizer.word_index

In [ ]:
vocab_length = len(tokenizer.word_index)+1
vocab_length

3495

In [ ]:
#convert review texts into sequences of integers
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Find max sequence length
max_seq_length=np.max(list(map(lambda x:len(x),X_train)))
max_seq_length

555

In [ ]:
#Pad sequences to be of uniform length
X_train = pad_sequences(X_train,maxlen=max_seq_length,padding='post')
X_test = pad_sequences(X_test,maxlen=max_seq_length,padding='post')
X_train

array([[322,  60,  33, ...,   0,   0,   0],
       [350,  14,   4, ...,   0,   0,   0],
       [ 34,  45,  29, ...,   0,   0,   0],
       ...,
       [ 34, 673,   5, ...,   0,   0,   0],
       [ 12, 105,   8, ...,   0,   0,   0],
       [  4,  30, 962, ...,   0,   0,   0]], dtype=int32)

In [ ]:
X_train.shape

(2016, 555)

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Embedding(
 input_dim=vocab_length,
 output_dim=128,
 input_length=max_seq_length
)(inputs)

x=tf.keras.layers.GRU(256, return_sequences=True,activation='tanh')(x)

x=tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model=tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

history = model.fit(
X_train,
y_train,
validation_split=0.2,
batch_size=32,
epochs=100,
callbacks=[
    tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=3,
      restore_best_weights=True
    )
])

Epoch 1/100
51/51 [==============================] - 16s 154ms/step - loss: 0.6058 - accuracy: 0.7091 - val_loss: 0.5564 - val_accuracy: 0.7376
Epoch 2/100
51/51 [==============================] - 5s 103ms/step - loss: 0.4904 - accuracy: 0.7934 - val_loss: 0.4720 - val_accuracy: 0.7797
Epoch 3/100
51/51 [==============================] - 5s 94ms/step - loss: 0.3001 - accuracy: 0.8797 - val_loss: 0.5202 - val_accuracy: 0.8020
Epoch 4/100
51/51 [==============================] - 6s 112ms/step - loss: 0.1882 - accuracy: 0.9268 - val_loss: 0.5414 - val_accuracy: 0.8218
Epoch 5/100
51/51 [==============================] - 3s 68ms/step - loss: 0.1266 - accuracy: 0.9498 - val_loss: 0.6323 - val_accuracy: 0.7896


In [ ]:
inputs

<KerasTensor: shape=(None, 555) dtype=float32 (created by layer 'input_1')>

In [ ]:
x

<KerasTensor: shape=(None, 142080) dtype=float32 (created by layer 'flatten')>

In [ ]:
model.evaluate(X_test,y_split)

16/16 [==============================] - 0s 22ms/step - loss: 0.4111 - accuracy: 0.8274


[0.4111465513706207, 0.8273809552192688]

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 555)]             0         
                                                                 
 embedding (Embedding)       (None, 555, 128)          447360    
                                                                 
 gru (GRU)                   (None, 555, 256)          296448    
                                                                 
 flatten (Flatten)           (None, 142080)            0         
                                                                 
 dense (Dense)               (None, 1)                 142081    
                                                                 
Total params: 885889 (3.38 MB)
Trainable params: 885889 (3.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
y_pred=model.predict(X_test)
y_pred
# y_split

16/16 [==============================] - 1s 13ms/step


array([[0.89789504],
       [0.9092571 ],
       [0.82659894],
       [0.61251163],
       [0.7869208 ],
       [0.945248  ],
       [0.8982179 ],
       [0.7540437 ],
       [0.67268807],
       [0.81415987],
       [0.6848276 ],
       [0.94791466],
       [0.84201837],
       [0.86990076],
       [0.94198793],
       [0.605664  ],
       [0.67200315],
       [0.73504555],
       [0.6433608 ],
       [0.8589711 ],
       [0.88974136],
       [0.14385742],
       [0.9759675 ],
       [0.8178142 ],
       [0.94357765],
       [0.25564703],
       [0.6718699 ],
       [0.37373486],
       [0.5608547 ],
       [0.86946684],
       [0.14937072],
       [0.4276358 ],
       [0.8043984 ],
       [0.88373446],
       [0.05459701],
       [0.9216521 ],
       [0.74143165],
       [0.3877716 ],
       [0.8458215 ],
       [0.9257776 ],
       [0.90089375],
       [0.4061641 ],
       [0.95520777],
       [0.7522928 ],
       [0.7496988 ],
       [0.95699316],
       [0.56318706],
       [0.984

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_split,y_pred,average='binary')

ValueError: ignored

In [ ]:
#importing test datset
test_df=pd.read_csv("/content/test.tsv",sep='\t')
test_df.head()

,date,time,id,reviewer name,reviewer address,contact number,variation,verified_reviews,feedback
0,29-Jul-18,16:47:11,6074908,Russell Hensley,"9059 Morales Run Suite 912\nSouth Charles, MH ...",6932290613,Heather Gray Fabric,I have the original Alexa and the Tap and so f...,1
1,28-Jul-18,17:49:58,1159008,Adam Baxter,"13728 Jones Mission Apt. 526\nSouth Tylerfurt,...",385-994-9458,Black Plus,Great product. More skills are needed. I love it.,1
2,30-Jul-18,16:46:29,1759753,Sarah Contreras,"7113 Pratt Plains\nPalmermouth, IN 02370",(738)354-8387,Configuration: Fire TV Stick,I will never buy anything Amazon makes again!T...,0
3,29-Jul-18,19:23:32,1861422,Tim Castillo,"93586 Cook Union Apt. 080\nMorganmouth, WI 51175",(294)877-6223,Black Plus,Not perfect but very good! A few times Alexa p...,1
4,28-Jul-18,16:15:25,3116018,Barry Young,69809 Harrington Meadows Apt. 256\nLake Courtn...,262-863-1391,Black Show,Kids like it for games. Showing the lyrics fo...,1


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              630 non-null    object
 1   time              630 non-null    object
 2   id                630 non-null    int64 
 3   reviewer name     630 non-null    object
 4   reviewer address  630 non-null    object
 5   contact number    630 non-null    object
 6   variation         630 non-null    object
 7   verified_reviews  630 non-null    object
 8   feedback          630 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 44.4+ KB


In [ ]:
x=test_df['verified_reviews']
x

0      I have the original Alexa and the Tap and so f...
1      Great product. More skills are needed. I love it.
2      I will never buy anything Amazon makes again!T...
3      Not perfect but very good! A few times Alexa p...
4      Kids like it for games.  Showing the lyrics fo...
                             ...                        
625    Best thing I have ever owned! Started with the...
626    I love being able to control lights, doors and...
627                                          works great
628    Purchased for my 83 year old mother-she loves ...
629         Fun so far...still learning how it all works
Name: verified_reviews, Length: 630, dtype: object

In [ ]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(x)


In [ ]:
x = tokenizer.texts_to_sequences(x)
# x

In [ ]:
# max_length=np.max(list(map(lambda t:len(t),x)))
# max_length

In [ ]:
x = pad_sequences(x,maxlen=max_seq_length,padding='post')
x

array([[  2,  15,   1, ...,   0,   0,   0],
       [ 16,  55,  41, ...,   0,   0,   0],
       [  2,  68, 273, ...,   0,   0,   0],
       ...,
       [ 33,  16,   0, ...,   0,   0,   0],
       [181,   9,   7, ...,   0,   0,   0],
       [ 91,  22,  92, ...,   0,   0,   0]], dtype=int32)

In [ ]:
y = model.predict(x)
y
# x.shape

20/20 [==============================] - 0s 18ms/step


array([[0.55967915],
       [0.88952136],
       [0.2602657 ],
       [0.20993729],
       [0.8246002 ],
       [0.55967915],
       [0.8659087 ],
       [0.9045516 ],
       [0.22598016],
       [0.5209702 ],
       [0.7496988 ],
       [0.9377088 ],
       [0.6378636 ],
       [0.64026386],
       [0.9171814 ],
       [0.87285954],
       [0.4008287 ],
       [0.96677464],
       [0.17847472],
       [0.08022811],
       [0.8695391 ],
       [0.84045804],
       [0.82333183],
       [0.56318706],
       [0.82566917],
       [0.8590554 ],
       [0.17432924],
       [0.7096747 ],
       [0.7854894 ],
       [0.4031838 ],
       [0.7474384 ],
       [0.5708589 ],
       [0.8401814 ],
       [0.79841727],
       [0.8830206 ],
       [0.66955113],
       [0.85052395],
       [0.9114445 ],
       [0.7976942 ],
       [0.92796123],
       [0.92659384],
       [0.09511905],
       [0.7823776 ],
       [0.9176939 ],
       [0.3168331 ],
       [0.88645774],
       [0.6640168 ],
       [0.713

In [ ]:
def scale_prediction(pred):
    return int(pred * 4 + 1)

# Apply the scaling function to your predictions
scaled_predictions = [scale_prediction(pred) for pred in y.flatten()]
scaled_predictions = np.clip(scaled_predictions, 1, 5)  # Ensure predictions are within the 1-5 range

# Create the submission DataFrame
data = {'id': test_df['id'].values, 'target': scaled_predictions}
submission_df = pd.DataFrame(data)
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df

,id,target
0,6074908,3
1,1159008,4
2,1759753,2
3,1861422,1
4,3116018,4
...,...,...
625,6856853,4
626,774782,4
627,1194506,4
628,4145653,4


#####

In [ ]:
data = {'id': test_df['id'].values, 'target': y.flatten()}
submission_df = pd.DataFrame(data)
submission_df


,id,target
0,6074908,0.559679
1,1159008,0.889521
2,1759753,0.260266
3,1861422,0.209937
4,3116018,0.824600
...,...,...
625,6856853,0.947897
626,774782,0.964006
627,1194506,0.897895
628,4145653,0.846981


In [ ]:
for i in range(630):
  a = submission_df.target[i]
  if a>0 and a<=0.2:
    submission_df.target[i] = 1
  elif a>0.2 and a<=0.4:
    submission_df.target[i] = 2
  elif a>0.4 and a<=0.6:
    submission_df.target[i] = 3
  elif a>0.6 and a<=0.8:
    submission_df.target[i] = 4
  elif a>0.8 and a<=1:
    submission_df.target[i] = 5

submission_df

<ipython-input-32-866955a1b63c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df.target[i] = 3
<ipython-input-32-866955a1b63c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df.target[i] = 5
<ipython-input-32-866955a1b63c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df.target[i] = 2
<ipython-input-32-866955a1b63c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,id,target
0,6074908,3.0
1,1159008,5.0
2,1759753,2.0
3,1861422,2.0
4,3116018,5.0
...,...,...
625,6856853,5.0
626,774782,5.0
627,1194506,5.0
628,4145653,5.0


In [ ]:
submission_df.to_csv('submission.csv', index=False)